<center> <h1> Restoration of underwater images </h1> <br> <h3> <br> 4MEO Computer vision 2</h3> </center>

# I. Introduction

### 1. Import libraries

In [764]:
import numpy as np
import scipy as sc
import cv2 as cv
import math
import matplotlib.image as mpimg
import matplotlib.pylab as plt

# II. Compute inputs

## 1. Show function

In [765]:
def show(final):
    cv.imshow('White balanced', final)
    cv.waitKey(0)
    cv.destroyAllWindows()
    for i in range (1,5):
        cv.waitKey(1)

## 2. White balance function definition 

In [766]:
def white_balance(img):
    result = cv.cvtColor(img, cv.COLOR_BGR2LAB)
    avg_a = np.average(result[:, :, 1])
    avg_b = np.average(result[:, :, 2])
    result[:, :, 1] = result[:, :, 1] - ((avg_a - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result[:, :, 2] = result[:, :, 2] - ((avg_b - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result = cv.cvtColor(result, cv.COLOR_LAB2BGR)
    return result

## 3. Enhance contrast function definition 

In [767]:
def enhance_contrast(im):
    rgb_img = im

    # convert from RGB color-space to YCrCb
    ycrcb_img = cv.cvtColor(rgb_img, cv.COLOR_BGR2YCrCb)

    # equalize the histogram of the Y channel
    ycrcb_img[:, :, 0] = cv.equalizeHist(ycrcb_img[:, :, 0])

    # convert back to RGB color-space from YCrCb
    equalized_img = cv.cvtColor(ycrcb_img, cv.COLOR_YCrCb2BGR)
    return equalized_img

## 4. Luminance weight map

In [768]:
def luminance_weight_map(img):
    im = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    L = np.mean(im,axis=2) #mean for each axes
    lumWeightMap = np.sqrt((1/3)*np.square((im[:,:,0]-L))+np.square((im[:,:,1]-L))+np.square((im[:,:,2]-L)))
    return lumWeightMap

## 5. Chromatic weight map

In [769]:
def chromatic_weight_map(img):
    hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    saturationValue = hsv[:,:,1]; 
    saturationMax = 1
    sigma = 0.3
    chromaticWeightmap = np.exp( -1 * ((np.square(saturationValue - saturationMax)) / (2*np.square(sigma))) )
    return chromaticWeightmap

## 6. Saliency weight map

In [770]:
def saliency_weight_map(img):
    saliency = cv.saliency.StaticSaliencySpectralResidual_create()
    (success, saliencyMap) = saliency.computeSaliency(img)
    saliencyMap = (saliencyMap * 255).astype("uint8")
    return saliencyMap

## 7. Generate Gaussian

In [771]:
def generate_gaussian(map):
    G = map.copy()
    gaussianPyramid = [G]
    for i in range(5):
        rows, cols = G.shape
        G = cv.pyrDown(G, dstsize=(cols//2, rows//2))
        gaussianPyramid.append(G)
    return gaussianPyramid

## 8. Generate Laplacian

In [772]:
def generate_laplacian(gaussianPyramid):
    laplacianPyramid = [gaussianPyramid[-1]]
    for i in range(5,0,-1):
        rows, cols = gaussianPyramid[i-1].shape
        GE = cv.pyrUp(gaussianPyramid[i], dstsize=(cols, rows))
        L = cv.subtract(gaussianPyramid[i-1],GE)
        laplacianPyramid.append(L)
    return laplacianPyramid

## Output

In [773]:
img = cv.imread('data/test4.png')

firstInput = white_balance(img)
secondInput = enhance_contrast(firstInput)

luminanceWeightmap1 = luminance_weight_map(firstInput)
chromaticWeightmap1 = chromatic_weight_map(firstInput)
saliencyWeightmap1 = saliency_weight_map(firstInput)
resultedWeightmap1 = np.multiply(np.multiply(luminanceWeightmap1,chromaticWeightmap1),saliencyWeightmap1)

luminanceWeightmap2 = luminance_weight_map(secondInput)
chromaticWeightmap2 = chromatic_weight_map(secondInput)
saliencyWeightmap2 = saliency_weight_map(secondInput)
resultedWeightmap2 = np.multiply(np.multiply(luminanceWeightmap2,chromaticWeightmap2),saliencyWeightmap2)

normaizedWeightmap1 = np.nan_to_num(np.true_divide(resultedWeightmap1,(resultedWeightmap1 + resultedWeightmap2)))
normaizedWeightmap2 = np.nan_to_num(np.true_divide(resultedWeightmap2,(resultedWeightmap1 + resultedWeightmap2)))

gaussianPyramid1 = generate_gaussian(normaizedWeightmap1)
gaussianPyramid2 = generate_gaussian(normaizedWeightmap2)

laplacianPyramid1 = generate_laplacian(gaussianPyramid1)
laplacianPyramid2 = generate_laplacian(gaussianPyramid2)

LS = []
for la,lb in zip(laplacianPyramid1,laplacianPyramid2):
    rows,cols = la.shape
    ls = np.hstack((la[:,0:cols//2], lb[:,cols//2:]))
    LS.append(ls)
# now reconstruct
ls_ = LS[0]
for i in range(1,5):
    rows, cols = LS[i].shape
    ls_ = cv.pyrUp(ls_, dstsize=(cols, rows))
    ls_ = cv.add(ls_, LS[i])

print(ls_)
final = np.hstack((img, firstInput, secondInput))

cv.imwrite('Pyramid_blending2.jpg',ls_)
show(final)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


[[0.9375     0.96875    0.875      ... 0.2109375  0.28125    0.1875    ]
 [0.625      0.73046875 0.8984375  ... 0.09765625 0.109375   0.07421875]
 [0.53627409 0.57366603 0.6861058  ... 0.2109375  0.09765625 0.08203125]
 ...
 [0.4921875  0.4140625  0.16796875 ... 0.59762739 0.34504727 0.57421875]
 [0.3515625  0.33984375 0.32421875 ... 0.6969931  0.48046875 0.49609375]
 [0.0625     0.14453125 0.36328125 ... 0.6423056  0.64453125 0.5625    ]]
